<a href="https://colab.research.google.com/github/amanichopra/rl-based-artificial-pancreas/blob/main/d3rlpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup rendering dependencies for Google Colaboratory.

In [ ]:
!apt-get install -y xvfb ffmpeg > /dev/null 2>&1
!pip install pyvirtualdisplay pygame moviepy > /dev/null 2>&1

Install d3rlpy!

In [ ]:
!pip3 show d3rlpy

Name: d3rlpy
Version: 2.3.0
Summary: An offline deep reinforcement learning library
Home-page: https://github.com/takuseno/d3rlpy
Author: Takuma Seno
Author-email: takuma.seno@gmail.com
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: click, colorama, dataclasses-json, gym, gymnasium, h5py, structlog, torch, tqdm, typing-extensions
Required-by: 


Setup dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

DATA_PATH = 'drive/MyDrive/School/MS/Coursework/Fall 2023/Reinforcement Learning/Project/transition_model_90_min_history_rewards.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv(DATA_PATH, index_col=0)
df.head()

,BG_t0,BG_t1,BG_t2,BG_t3,BG_t4,BG_t5,BG_t6,BG_t7,BG_t8,BG_t9,...,IOB_t28,IOB_t29,IOB_t30,IOB_t31,IOB_t32,IOB_t33,IOB_t34,IOB_t35,simple_reward,magni_reward
0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,111.0,118.0,...,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0.568121,0.580412,0,-10.211281
1,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,111.0,...,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0.568121,0,-12.579462
2,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,107.0,...,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0.555829,0,-12.139707
3,120.0,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,101.0,...,0.458479,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0.543538,0,-12.800759
4,106.0,120.0,130.0,132.0,127.0,121.0,113.0,99.0,95.0,97.0,...,0.444016,0.458479,0.469787,0.471303,0.491872,0.502033,0.510965,0.531246,0,-14.602987


In [ ]:
from d3rlpy.dataset import MDPDataset
import numpy as np

feature_dim = int((max([int(col[4:]) for col in df.columns if 'BG' in col]) + 1) / 2)
s_cols = [f'BG_t{i}' for i in range(feature_dim)] + [f'HR_t{i}' for i in range(feature_dim)] + [f'IOB_t{i}' for i in range(feature_dim)]
a_col = 'InsulinDelivered'
r_col = 'magni_reward'

obs = df[s_cols].values
actions = df[a_col].values
rewards = df[r_col].values
terminals = np.random.randint(2, size=df.shape[0])

dataset = MDPDataset(obs, actions, rewards, terminals)

2023-12-05 01:12.57 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(54,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2023-12-05 01:12.57 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2023-12-05 01:12.57 [info     ] Action size has been automatically determined. action_size=12


In [ ]:
dataset

In [ ]:
from d3rlpy.algos import DQN
import d3rlpy

test_episodes = dataset.episodes[:10]

dqn = d3rlpy.algos.DQNConfig().create(device='cuda')

# prepare algorithm
# dqn.fit(
#     dataset,
#     n_steps=10000,
#     evaluators={
#         'td_error': d3rlpy.metrics.TDErrorEvaluator(test_episodes),
#         'value_scale': d3rlpy.metrics.AverageValueEstimationEvaluator(test_episodes)
#     },
#     # set FileAdapterFactory to save metrics as CSV files
#    logger_adapter=d3rlpy.logging.FileAdapterFactory(root_dir="d3rlpy_logs")
# )

for epoch, metrics in dqn.fit(
    dataset,
    n_steps=10000,
    evaluators={
        'td_error': d3rlpy.metrics.TDErrorEvaluator(test_episodes),
        'value_scale': d3rlpy.metrics.AverageValueEstimationEvaluator(test_episodes)
    },
    # set FileAdapterFactory to save metrics as CSV files
   logger_adapter=d3rlpy.logging.FileAdapterFactory(root_dir="d3rlpy_logs")
):
    print(epoch)


2023-12-05 01:25.27 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(54,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=12)
2023-12-05 01:25.27 [info     ] Directory is created at d3rlpy_logs/DQN_20231205012527
2023-12-05 01:25.27 [debug    ] Building models...
2023-12-05 01:25.27 [debug    ] Models have been built.
2023-12-05 01:25.27 [info     ] Parameters                     params={'observation_shape': [54], 'action_size': 12, 'config': {'type': 'dqn', 'params': {'batch_size': 32, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params': {'betas': [0.9, 0.999], 'eps': 1e-08, 'wei

Epoch 1/1:   0%|          | 0/10000 [00:00<?, ?it/s]

2023-12-05 01:26.32 [info     ] DQN_20231205012527: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.001975260591506958, 'time_algorithm_update': 0.004272715735435486, 'loss': 6.426356756305695, 'time_step': 0.006395246434211731, 'td_error': 22.802399592364537, 'value_scale': 2.121822476387024} step=10000
2023-12-05 01:26.32 [info     ] Model parameters are saved to d3rlpy_logs/DQN_20231205012527/model_10000.d3
1
